# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle

# import relevant functions/modules from the sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report, accuracy_score, precision_score, f1_score, recall_score

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV 


import xgboost as xgb

[nltk_data] Downloading package punkt to /Users/fernandes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/fernandes/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fernandes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../ETL-Pipeline-Preparation/InsertDatabaseName.db')
df = pd.read_sql_table("pipelines", con=engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.dtypes

id                         int64
message                   object
original                  object
genre                     object
related                    int64
request                    int64
offer                      int64
aid_related                int64
medical_help               int64
medical_products           int64
search_and_rescue          int64
security                   int64
military                   int64
child_alone                int64
water                      int64
food                       int64
shelter                    int64
clothing                   int64
money                      int64
missing_people             int64
refugees                   int64
death                      int64
other_aid                  int64
infrastructure_related     int64
transport                  int64
buildings                  int64
electricity                int64
tools                      int64
hospitals                  int64
shops                      int64
aid_center

In [5]:
X = df["message"]
y = df.iloc[:,4:]

In [6]:
print(X)

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26216, dtype: object


In [7]:
print(y.columns)

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')


In [8]:
X.shape, y.shape

((26216,), (26216, 36))

In [9]:
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    """Normalize, tokenize and stem text string
    
    Args:
    text: string. String containing message for processing
       
    Returns:
    stemmed: list of strings. List containing normalized and stemmed word tokens
    """
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline =  Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(warm_start=True))),
    ], verbose=True)

In [12]:
# Create the pipeline: xgb_pipeline
pipeline_xgb = Pipeline([
         ('ohe_onestep', CountVectorizer(tokenizer = tokenize)),
         ('xgb_model', xgb.XGBRegressor())
     ])

## 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [14]:
# Quick check to examine what we have got
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(19662,)
(6554,)
(19662, 36)
(6554, 36)


In [15]:
# Train classifier
pipeline.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=  10.0s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 7.7min


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f866ff5f320>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(warm_start=True)))],
         verbose=True)

In [16]:
# Fit the XGBregressor pipeline
pipeline_xgb.fit(X_train, y_train)

Pipeline(steps=[('ohe_onestep',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f866ff5f320>)),
                ('xgb_model',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=0, num_parallel_tree=1,
                              p

## 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
# Classifier Prediction
y_pred = pipeline.predict(X_test)

In [18]:
# Printing the classification report for each label
def model_report(y_test, y_pred):
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [19]:
model_report(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.75      0.29      0.42      1515
           1       0.82      0.97      0.89      4993
           2       0.37      0.43      0.40        46

    accuracy                           0.81      6554
   macro avg       0.65      0.56      0.57      6554
weighted avg       0.80      0.81      0.77      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5433
           1       0.91      0.45      0.60      1121

    accuracy                           0.90      6554
   macro avg       0.90      0.72      0.77      6554
weighted avg       0.90      0.90      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metr

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6130
           1       1.00      0.00      0.00       424

    accuracy                           0.94      6554
   macro avg       0.97      0.50      0.49      6554
weighted avg       0.94      0.94      0.90      6554

Feature 21: transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6256
           1       0.85      0.10      0.17       298

    accuracy                           0.96      6554
   macro avg       0.91      0.55      0.58      6554
weighted avg       0.95      0.96      0.94      6554

Feature 22: buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6209
           1       1.00      0.08      0.15       345

    accuracy                           0.95      6554
   macro avg       0.98      0.54      0.56      6554
weighted avg       0.95      0

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metr

In [20]:
# Prediction: XGB Regressor 
y_pred = pipeline_xgb.predict(X_test)

In [21]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print('MAE score is {}'.format(mae))

The model performance for testing set
--------------------------------------
RMSE is 0.19888784345199914
R2 score is 0.22420125506426614
MAE score is 0.07598837058144


## 6. Improve your model
Use grid search to find better parameters. 

In [22]:
# Show parameters for the pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f866ff5f320>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(warm_start=True)))],
 'verbose': True,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f866ff5f320>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(warm_start=True)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf

In [23]:
parameters = {
    'clf__estimator': (MultinomialNB(), 
                       RandomForestClassifier(warm_start=True))
}



In [24]:
cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=1, verbose=10)
cv

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f866ff5f320>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(warm_start=True)))],
                                verbose=True),
             n_jobs=1,
             param_grid={'clf__estimator': (MultinomialNB(),
                                            RandomForestClassifier(warm_start=True))},
             verbose=10)

In [25]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3; 1/2] START clf__estimator=MultinomialNB()..............................
[Pipeline] .............. (step 1 of 3) Processing vect, total=   6.8s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.4s
[CV 1/3; 1/2] END clf__estimator=MultinomialNB();, score=0.165 total time=  15.8s
[CV 2/3; 1/2] START clf__estimator=MultinomialNB()..............................
[Pipeline] .............. (step 1 of 3) Processing vect, total=   6.8s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.2s
[CV 2/3; 1/2] END clf__estimator=MultinomialNB();, score=0.162 total time=   9.8s
[CV 3/3; 1/2] START clf__estimator=MultinomialNB()..............................
[Pipeline] .............. (step 1 of 3) Processing vect, total=   7.1s
[Pipeline] ............. (step 2 of 3

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f866ff5f320>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(warm_start=True)))],
                                verbose=True),
             n_jobs=1,
             param_grid={'clf__estimator': (MultinomialNB(),
                                            RandomForestClassifier(warm_start=True))},
             verbose=10)

In [26]:
cv.best_params_

{'clf__estimator': RandomForestClassifier(warm_start=True)}

## 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
# Predicting using the first tuned model 
ypred_test = cv.predict(X_test)

In [28]:
# Predicting using the first tuned model 
model_report(y_test, ypred_test)

Feature 1: related
              precision    recall  f1-score   support

           0       0.74      0.29      0.42      1515
           1       0.82      0.97      0.89      4993
           2       0.44      0.48      0.46        46

    accuracy                           0.81      6554
   macro avg       0.67      0.58      0.59      6554
weighted avg       0.80      0.81      0.77      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5433
           1       0.90      0.46      0.61      1121

    accuracy                           0.90      6554
   macro avg       0.90      0.72      0.77      6554
weighted avg       0.90      0.90      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metr

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6462
           1       0.75      0.07      0.12        92

    accuracy                           0.99      6554
   macro avg       0.87      0.53      0.56      6554
weighted avg       0.98      0.99      0.98      6554

Feature 15: money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6414
           1       1.00      0.01      0.03       140

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.98      0.98      0.97      6554

Feature 16: missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6472
           1       0.00      0.00      0.00        82

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metr

              precision    recall  f1-score   support

           0       0.87      0.95      0.91      4748
           1       0.83      0.64      0.72      1806

    accuracy                           0.86      6554
   macro avg       0.85      0.79      0.82      6554
weighted avg       0.86      0.86      0.86      6554

Feature 30: floods
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6026
           1       0.89      0.37      0.52       528

    accuracy                           0.95      6554
   macro avg       0.92      0.68      0.74      6554
weighted avg       0.94      0.95      0.93      6554

Feature 31: storm
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      5968
           1       0.74      0.40      0.52       586

    accuracy                           0.93      6554
   macro avg       0.84      0.70      0.74      6554
weighted avg       0.93      0.93    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
improved_pipeline =  Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(warm_start=True))),
    ], verbose=True)

In [30]:
improved_pipeline

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f866ff5f320>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(warm_start=True)))],
         verbose=True)

In [34]:
improved_pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f866ff5f320>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(warm_start=True)))],
 'verbose': True,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f866ff5f320>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(warm_start=True)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf

In [38]:
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4],
    }

In [39]:
model = GridSearchCV(improved_pipeline, parameters)

In [ ]:
#Use improved pipeline
model.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=   7.5s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 2.8min
[Pipeline] .............. (step 1 of 3) Processing vect, total=   6.6s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 3.0min
[Pipeline] .............. (step 1 of 3) Processing vect, total=  10.9s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 3.4min
[Pipeline] .............. (step 1 of 3) Processing vect, total=  12.8s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 3.2min
[Pipeline] .............. (step 1 of 3) Processing vect, total=   8.9s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipel

In [ ]:
# ypred_train = pipeline_imp.predict(X_train)
ypred_test_improved = model.predict(X_test)

In [ ]:
# Predicting using the first tuned model 
model_report(y_test, ypred_test_improved)

### 9. Export your model as a pickle file

In [ ]:
# Create a pickle file for the model
file_name = 'classifier-000.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.